# Demo notebook for stats_can package

API documentation for StatsCan can be found here: https://www.statcan.gc.ca/eng/developers/wds

If you're looking for Table/Vector IDs to use in the app you can find them through this: https://www150.statcan.gc.ca/n1/en/type/data

Anaconda package here: https://anaconda.org/ian.e.preston/stats_can

This notebook will provide an overview of the key functionality of the stats_can python package

In [1]:
import stats_can
import datetime as dt
import pandas as pd

The first thing we can do is retrieve a list of tables/vectors that have been updated today:
Note that the changed series list returns a huuuuuge list and takes forever to run

In [2]:
changed_series = stats_can.get_changed_series_list()
# show just the first few
changed_series[0:min(2, len(changed_series))]

[{'responseStatusCode': 0,
  'vectorId': 111666243,
  'productId': 33100036,
  'coordinate': '1.20.0.0.0.0.0.0.0.0',
  'releaseTime': '2018-08-22T08:30'},
 {'responseStatusCode': 0,
  'vectorId': 111666232,
  'productId': 33100036,
  'coordinate': '1.9.0.0.0.0.0.0.0.0',
  'releaseTime': '2018-08-22T08:30'}]

In [3]:
# This takes a date argument or defaults to today
changed_tables = stats_can.get_changed_cube_list()
# Show just the first few
changed_tables[0:min(2, len(changed_tables))]

[{'responseStatusCode': 0,
  'productId': 33100036,
  'releaseTime': '2018-08-22T08:30'},
 {'responseStatusCode': 0,
  'productId': 10100139,
  'releaseTime': '2018-08-22T08:30'}]

MetaData on tables can also be retrieved:
Note that the package makes some minimal efforts to parse vector/cube/table names into the format accepted by StatsCan

In [4]:
table_name = '10-10-0106-01'
table_meta = stats_can.get_cube_metadata(table_name)
table_meta[0]['cubeTitleEn']

'Consumer Price Index (CPI) statistics, alternative measures, unadjusted and seasonally adjusted, Bank of Canada'

You can perform a similar operation for individual or lists of vectors

In [5]:
vectors = ['v74804', 'v41692457']
series_info = stats_can.get_series_info_from_vector(vectors)
series_info[0]['SeriesTitleEn']

'Alberta;All-items excluding energy'

Tables for vectors is mostly a helper method, if you have a list of vectors that you're interested in, it will return a dictionary mapping what tables those vectors are in, as well as return a list of all tables needed to retrieve those vectors

In [6]:
stats_can.get_tables_for_vectors(vectors)

{41692457: '18100004',
 74804: '23100216',
 'all_tables': ['23100216', '18100004']}

Similar deal for table subsets for vectors, it returns a dictionary with keys of table names mapped to a list of vectors contained in those tables

In [7]:
stats_can.table_subsets_from_vectors(vectors)

{'23100216': [74804], '18100004': [41692457]}

Vectors to df will take a list of vectors and either an argument specifying the number of most recent periods to retrieve, or a start and end date for the period in which the data was released. Note that release date is not the same as the reference period, which is the date index you'll see in the returned dataframe

In [8]:
df_n = stats_can.vectors_to_df(vectors, periods=5)
df_n

,v41692457,v74804
refPer,,
2018-01-01,NaN,291408.0
2018-02-01,NaN,263194.0
2018-03-01,137.8,314121.0
2018-04-01,137.7,301073.0
2018-05-01,138.0,306822.0
2018-06-01,138.2,NaN
2018-07-01,138.7,NaN


In [9]:
df_d = stats_can.vectors_to_df(vectors, start_release_date=dt.date(2018, 1, 1), end_release_date=dt.date(2018, 8, 20))
df_d.head()

,v41692457,v74804
refPer,,
2013-01-01,124.5,NaN
2013-02-01,125.1,NaN
2013-03-01,125.5,NaN
2013-04-01,125.8,NaN
2013-05-01,126.3,NaN


The package has some methods exposed to download zipped csv files of entire tables and read them into dataframes, but I've got an updated implementation that just uses those methods to download tables and read them into an hdf5 file (https://support.hdfgroup.org/HDF5/) which ends up being way faster. For one particularly large table my machine ran about 2.5 minutes to load a zipped csv of the table into a dataframe, whereas reading it from hdf5 took 30 seconds. If you're going to be accessing the tables with an regularity it's totally worth using hdf5.

These are the main resources I used to figure out hdf5 for this

https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/

https://realpython.com/fast-flexible-pandas/#prevent-reprocessing-with-hdfstore

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_hdf.html

In [10]:
# can also include arguments for where to store the h5 file and what to name it
tables = ['27100022', '18100204']
stats_can.tables_to_h5(tables)

The h5_update_tables function will compare the metadata associated with tables in an h5 file, compare it to the metadata from the StatsCan website, download any out of date tables and read them into the h5 file. There's no output so I won't call the method here, but the syntax is:
stats_can.h5_update_tables(h5file='name_of_file.h5', path='path_to_h5_file')

Having added a couple tables into the h5 file we can read them out using the table_from_h5 method.

In [11]:
df_tbl = stats_can.table_from_h5('27100022')
df_tbl.head()

,REF_DATE,GEO,DGUID,Performing sector,Occupational category,Type of science,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2000-01-01,Canada,2016A000011124,Total performing sector,Total personnel,Total sciences,Number,223,units,0,v54329358,1.1.1.1,167940.0,NaN,NaN,NaN,0
1,2000-01-01,Canada,2016A000011124,Total performing sector,Total personnel,Natural sciences and engineering,Number,223,units,0,v54329359,1.1.1.2,147290.0,NaN,NaN,t,0
2,2000-01-01,Canada,2016A000011124,Total performing sector,Total personnel,Social sciences and humanities,Number,223,units,0,v54329360,1.1.1.3,20650.0,NaN,NaN,t,0
3,2000-01-01,Canada,2016A000011124,Total performing sector,Researchers,Total sciences,Number,223,units,0,v54329361,1.1.2.1,107900.0,NaN,NaN,NaN,0
4,2000-01-01,Canada,2016A000011124,Total performing sector,Researchers,Natural sciences and engineering,Number,223,units,0,v54329362,1.1.2.2,91590.0,NaN,NaN,t,0


The local metadata can be retrieved in a similar manner

In [12]:
stats_can.metadata_from_h5('27100022')[0]['cubeEndDate']

'2015-01-01'

Can also see what keys are in the file, again helpers for json and df keys are on my todo

In [13]:
stats_can.h5_included_keys()

['json_18100204', 'json_27100022', 'table_18100204', 'table_27100022']

You can also tell it to check the metadata in the locally stored tables, compare it against the StatsCan website metadata, and then update any tables, or a subset of tables that have fresh data:

In [14]:
stats_can.h5_update_tables(tables='18100204') # will only check this one table
stats_can.h5_update_tables()  # will check all tables in the file and update the appropriate ones

[]